# Keras + Tensorflow

Tf очень мощная и крутая штука, но кодить на нем большие и сложные архитектуры не всегда оправданное занятие.

На этот случай существует множество высокоуровневых фреймворков, под капотом которых работает Tensorflow:
* [TFSlim](https://research.googleblog.com/2016/08/tf-slim-high-level-library-to-define.html)
* [TFLearn](https://www.tensorflow.org/api_docs/python/tf/contrib/learn)
* [Sonnet (deepMind)](https://github.com/deepmind/sonnet)
* Keras

Мы немного посмотрим на Keras

In [ ]:
# Warning!
# ща тут данные начнут скачиваться, если их еще нет

import numpy as np
from mnist import load_dataset
import keras
X_train, y_train, X_val, y_val, X_test, y_test = load_dataset()
y_train,y_val,y_test = list(map(keras.utils.np_utils.to_categorical,[y_train,y_val,y_test]))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.imshow(X_train[0,0]);

# Еще одно маленькое предисловие

Keras позволяет имплементировать архитектуру deep-learning моделей такими высокоуровневыми абстракциями как `Dense layer with nonlinearity`, `dropout` или `batch normalization`.
Keras даже берет на себя ответственность за правильную работу с `dropout` во время оценивания качества модели.

Использовать один только Keras удобно, если речь идет о построении чего-то не очень замудренного вроде классификатора изображений.

Но если архитектура предполагает совместное использование весовых матриц или `adversarial training` (и прочее похожее), то использование одного только high-level Keras'а становится затруднительным.

In [ ]:
import tensorflow as tf
gpu_options = tf.GPUOptions(allow_growth=True,per_process_gpu_memory_fraction=0.1)
s = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

In [ ]:
import keras
from keras.models import Sequential
import keras.layers as ll

model = Sequential(name="cnn")

model.add(ll.InputLayer([1,28,28]))

model.add(ll.Flatten())

#network body
model.add(ll.Dense(25))
model.add(ll.Activation('linear'))

model.add(ll.Dropout(0.9))

model.add(ll.Dense(25))
model.add(ll.Activation('linear'))

#output layer: 10 neurons (for each class) with softmax
model.add(ll.Dense(10,activation='softmax'))

model.compile("adam","categorical_crossentropy",metrics=["accuracy"])

In [ ]:
model.summary()

### Model interface

Модели в `Keras` используют тот же интерфейс fit/predict, что и модели в __Scikit-learn__.
Давайте посмотрим.

In [ ]:
# fit(X,y) с автоматическим логгированием.
model.fit(X_train, y_train, validation_data=(X_val,y_val), epochs=5)

In [ ]:
# посчитаем вероятности P(y|x)
model.predict_proba(X_val[:2])

In [ ]:
#Save trained weights
model.save("weights.h5")

In [ ]:
print("\nLoss, Accuracy = ", model.evaluate(X_test,y_test))

### Упс..!

Кажется, что с текущей архитектурой модель крайне неэффективна. Она даже не может побить результата линейной модели (val_acc ~0.92).

Догадываешься почему?

Здесь специально сделано пару ошибочек.

Одна ошибочка мешает сетке выучить нелинейные зависимости, другая -- не дает ей выучить необходимое количество данных.

Попробуй зафиксить обе ошибочки и поиграй с архитектурой пока не побьешь бейзлайны снизу.

In [ ]:
# Test score...
test_predictions = model.predict_proba(X_test).argmax(axis=-1)
test_answers = y_test.argmax(axis=-1)

test_accuracy = np.mean(test_predictions==test_answers)

print("\nTest accuracy: {} %".format(test_accuracy*100))

assert test_accuracy>=0.92,"Logistic regression can do better!"
assert test_accuracy>=0.975,"Your network can do better!"
print("Great job!")

## Keras + tensorboard

Помнишь интерактивные графики от Tensorboard один ноутбук назад?

Дело в том, что Keras может использовать Tensorboard, чтобы показать полезную информацию о ходе обучения.
Давай посмотрим.

In [ ]:
model = Sequential(name="cnn")

model.add(ll.InputLayer([1,28,28]))
model.add(ll.Flatten())


<Your architecture here>


#output layer: 10 neurons for each class with softmax
model.add(ll.Dense(10,activation='softmax'))

model.compile("adam","categorical_crossentropy",metrics=["accuracy"])


In [ ]:
from keras.callbacks import TensorBoard
model.fit(X_train, y_train, validation_data=(X_val,y_val),epochs=1,
          callbacks=[TensorBoard("./logs")])

In [ ]:
import os
port = 6000 + os.getuid()
print("Port: %d" % port)
# !killall tensorboard
os.system("tensorboard --logdir=./logs --port=%d &" % port)

# Quest For A Better Network
Несколько советов о том, что ты можешь сделать со своей нейронкой;

Советы очень общего характера и не относятся напрямую к решаемой в тетрадке задачке, но в целом они оч полезны.

 * __Размер имеет значение__
   * MOAR neurons, 
   * MOAR layers, ([docs](https://keras.io/))

   * Нелинейности -- функции активации могут быть разными:
     * tanh, relu, leaky relu, etc
   * Чем больше сетка, тем больше эпох ей нужно чтобы обучиться. Из этого вывод -- не стоит выбрасывать свою нейронку сразу, если после 5-ти эпох она все еще не обогнала бейзлайн (SPOILER!: на самом деле бейзлайн можно обогнать и за 3 эпохи:) )
   * Ph'nglui mglw'nafh Cthulhu R'lyeh wgah'nagl fhtagn!


 * __Convolution layers__
   * Это __must have__ пока у вас нет других супер идей
   * `keras.layers.Conv2D`
   * Warning! Сверточные сети тренируются довольно долго, если не использовать gpu. Это нормально.
     * Если ты на cpu, то рекомендуется использовать что-то не слишком мощное.
     * В ином варианте можно запустить сеть обучаться на ночь и проверить результат утром.
     * Сделайте разумные оценки размера слоев. Первоначальная свертка с 128 нейронами, вероятно, является оверкиллом.
     * __Чтобы уменьшить количество вычислений__ на порядок жертвуя точностью, попробуй использовать параметр __stride__.
     Пример: stride=2 вместо stride=1
     делает свертку сдвигаясь не на одну ячейку, а не 2, таким образом на выходе свертка примерно в 4 раза меньше.
 
   * Еще куча всяких слоев и архитектур может быть найдена тут
     * batch normalization, pooling, etc
     * Docs - https://keras.io/layers
     
     

 * __Early Stopping__
   * Тренить 100 эпох несмотря ни на что это плохая идея.
   * Какие-то сети сходятся за 5 эпох, другие через 500 эпох.
   * Путь к успеху: остановка, когда скор на валидации через 10 итераций не становится лучше достигнутого максимума
     



 * __Игры с оптимизацией__ - 
   * rmsprop, nesterov_momentum, adam, adagrad и другие.
     * Возможно имеет смысл потюнить learning rate/momentum и др параметры обучения: batch size, number of epochs..
 
 
 
 * __BatchNormalization__ FTW!(for the win)
     * `keras.layers.normalization.BatchNormalization`




 * __Регуляризация__ чтобы избежать переобучения
   * Добавьте L2 нормализацию весов модели в функцию лосса
     * Смотреть - https://keras.io/regularizers/
   * Dropout - хотя в данном примере он уже есть
     * `keras.layers.Dropout`   
     * Действительно ли дропаут делает сетку лучше? Проверить.
   
   
 * __Data augmemntation__ - запросто получить в 5 раз больший набор данных? -- иногда это возможно 
   * https://keras.io/preprocessing/image/
   * Zoom-in+slice = move
   * Rotate+zoom(для удаления черных полосок?)
   * Любые другие манипуляции
   * Простой путь к этому (using PIL/Image): 
     * ```from scipy.misc import imrotate,imresize```
     * и немного slicing'а
   * Оставайтесь реалистичными.
   Как правило, нет смысла переворачивать собак вверх ногами, поскольку это не так, как вы обычно их видите.